In [0]:
%sql
USE CATALOG workspace;
USE pl_delivery_analysis;

-- ===========================================================================
-- LOG INICIAL
-- ===========================================================================

INSERT INTO log_processamento 
(processo, etapa, status, timestamp, detalhes)
VALUES ('etl_quality', 'inicio', 'iniciado', CURRENT_TIMESTAMP(), 'Iniciando ETL e Quality Check');

SELECT ' INICIANDO ETL COMPLETO + QUALITY CHECK...' as status;

-- ===========================================================================
-- 1. CAMADA BRONZE
-- ===========================================================================

SELECT ' INICIANDO CAMADA BRONZE...' as status;

-- ===========================================================================
-- CAMADA BRONZE - INGESTÃO
-- ===========================================================================

-- Registrar início Bronze
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
VALUES ('bronze', 'ingestao_inicio', 'iniciado', CURRENT_TIMESTAMP(), 'Iniciando ingestão Bronze');

-- Criar volume se não existir
CREATE VOLUME IF NOT EXISTS raw_data_volume;

-- -------------------------------------------------------------
-- A. FATO PRINCIPAL (ORDERS)
-- -------------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_orders_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_orders_bronze (
  order_id STRING,
  created_at STRING,
  subtotal STRING,
  delivery_fee STRING,
  store_id STRING
);

COPY INTO pl_delivery_analysis.tbl_fact_orders_bronze
FROM (
  SELECT
    order_id,
    order_moment_created AS created_at,
    order_amount AS subtotal,
    order_delivery_fee AS delivery_fee,
    store_id
  FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/orders.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ','); 

-- -----------------------------------------------------------
-- B. FATO LOGÍSTICO (DELIVERIES) 
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_deliveries_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_deliveries_bronze (
    order_id STRING,
    delivery_status STRING,
    driver_id STRING
);
COPY INTO pl_delivery_analysis.tbl_fact_deliveries_bronze
FROM (
    SELECT
        delivery_order_id AS order_id,
        delivery_status,
        driver_id
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/deliveries.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ','); 

-- -----------------------------------------------------------
-- C. FATO FINANCEIRO (PAYMENTS)
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_payments_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_payments_bronze (
    order_id STRING,
    payment_method STRING,
    payment_amount STRING
);
COPY INTO pl_delivery_analysis.tbl_fact_payments_bronze
FROM (
    SELECT
        payment_order_id AS order_id,
        payment_method,
        payment_amount
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/payments.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');

-- -----------------------------------------------------------
-- D. DIMENSÃO LOJA (STORES)
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_stores_bronze;
CREATE TABLE pl_delivery_analysis.tbl_dim_stores_bronze (
    store_id STRING,
    store_segment STRING,
    hub_id STRING 
);
COPY INTO pl_delivery_analysis.tbl_dim_stores_bronze
FROM (
    SELECT
        store_id,
        store_segment,
        hub_id
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/stores.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');

-- -----------------------------------------------------------
-- E. DIMENSÃO HUB (HUBS) 
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_hubs_bronze;
CREATE TABLE pl_delivery_analysis.tbl_dim_hubs_bronze (
    hub_id STRING,
    city STRING
);
COPY INTO pl_delivery_analysis.tbl_dim_hubs_bronze
FROM (
    SELECT
        hub_id,
        hub_city AS city 
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/hubs.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');

-- Verificação da Camada Bronze
SELECT 
  'Bronze - Orders' as tabela, COUNT(*) as total 
FROM pl_delivery_analysis.tbl_fact_orders_bronze
UNION ALL
SELECT 
  'Bronze - Deliveries', COUNT(*) 
FROM pl_delivery_analysis.tbl_fact_deliveries_bronze
UNION ALL
SELECT 
  'Bronze - Payments', COUNT(*) 
FROM pl_delivery_analysis.tbl_fact_payments_bronze
UNION ALL
SELECT 
  'Bronze - Stores', COUNT(*) 
FROM pl_delivery_analysis.tbl_dim_stores_bronze
UNION ALL
SELECT 
  'Bronze - Hubs', COUNT(*) 
FROM pl_delivery_analysis.tbl_dim_hubs_bronze;

-- Registrar fim Bronze
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
SELECT 
    'bronze', 
    'ingestao_fim', 
    'sucesso', 
    CURRENT_TIMESTAMP(), 
    CONCAT('Bronze carregado: ', 
           (SELECT COUNT(*) FROM tbl_fact_orders_bronze), ' orders, ',
           (SELECT COUNT(*) FROM tbl_fact_deliveries_bronze), ' deliveries, ',
           (SELECT COUNT(*) FROM tbl_fact_payments_bronze), ' payments')
FROM (SELECT 1) t;